 <font face="Verdana, cursive, sans-serif" >
 <H1><center>Map Visualization with leaflets.js and mapbox.js</center></H1>

<font face="Verdana, cursive, sans-serif" >

<p>This notebook explains how to overlay data on a geo shape file. There 2 parts in this program. The first part explains how to add new attributes from a csv file to a shape-json file. The second part explain how the pre-coded javascript works</p>

<b>Self-paced Learning Resources</b>
<ul>
    <li><a href="https://www.youtube.com/watch?v=SCAqyPfwzcU">Complete leaflets tutorial</a></li>
    <li><a href="http://leafletjs.com/examples/choropleth/">Leaflet.js - Interactive Choropleth</a></li>
    <li><a href="https://www.mapbox.com/help/tutorials/">Mapbox turorials</a></li>
    <li><a href="https://www.mapbox.com/api-documentation/#maps">Mapbox style</a></li>
    <li><a href="https://github.com/codeforamerica/click_that_hood/tree/master/public/data">Country Polygon Files</a></li>
</ul>


 <font face="Verdana, cursive, sans-serif" >
 <br>
 <H2>Part 1 : Merge  GeoJson with external Data </H2>
<br>
 <img src="./image/part1.png" >

<font face="Verdana, cursive, sans-serif" >
<h3>Importing packages</h3>

The json package is used for processing json format files in python
https://docs.python.org/2/library/json.html

In [1]:
import pandas as pd
import numpy as np
import json

<font face="Verdana, cursive, sans-serif" >
<h3>Define input / output files</h3>

In [2]:
data_csvfile = './data/ext_province_th.csv'
shape_jsonfile = './data/thailand.json'
output_js = './data/th_geo.js'

<font face="Verdana, cursive, sans-serif" >
<h3>Depends on the csv file, carefully identify the key needed to use for merging</h3>
<p>
<img src="./image/th_key.png" >

Note that the key values should be: <b>cleaned, upcased and no spaces</b>

In [3]:
#this is the key column to merge btw the shape file and the external data
key='provinces'

<font face="Verdana, cursive, sans-serif" >
<h3>Read and pre-process the csv data</h3>
<p>Please note that the csv data used in this program a a mockup data</p>

In [4]:
data_csv = pd.read_csv(data_csvfile)
data_csv.fillna(0,inplace=True)
data_csv.head()

,provinces,total_pop,avg_income,mkt_penlt
0,MAEHONGSON,83000,8654,5.9
1,CHUMPHON,40000,2691,52.5
2,NAKHONSITHAMMARAT,59000,4843,61.5
3,PHUKET,4000,2875,89.3
4,PHANGNGA,73000,7864,38.4


In [5]:
# Pre-processing (convert numeric to same serializable format )
for col in data_csv.columns:
    if data_csv[col].dtype == 'int64':
        data_csv[col] = data_csv[col].astype(np.float64)

<font face="Verdana, cursive, sans-serif" >
<h3>Read and pre-process the shape file</h3>

<p>Depends on the shape file, carefully identify the features-names needed to use for processing and merging
<p>
<img src="./image/th_shapejson.png" >

In [6]:
with open(shape_jsonfile, "r") as jsonFile:
    data_shapejson = json.load(jsonFile)

#grab all nodes, put it in a dictionary for later processing 
geo_map = {}
for f in data_shapejson['features']:
    k = f['properties']['name']
    v = k.replace(" ","").upper()
    geo_map[k] = v 

# It is good to check the number of nodes that we have, for Thailand, it should be 77 because we have 77 provinces
print(len(geo_map))
    

77


<font face="Verdana, cursive, sans-serif" >
<h3>For each item in csv, add a new node 'info' at the corresponding key</h3>

In [7]:
for f in data_shapejson['features']:
    info = {}
    name = f['properties']['name']
    info['name'] = name
    
    if not name in geo_map.keys():
        continue
    
    value_df = data_csv[data_csv[key]==geo_map[name]].reset_index(drop=True)

    for col in [x for x in data_csv.columns.values if x not in [key]]:
        info[col] = value_df[col].values[0]
    
    f['info'] = info

 

In [8]:
print(data_shapejson['features'][0]['properties']['name'])
print(data_shapejson['features'][0]['info']['total_pop'])
print(data_shapejson['features'][0]['info']['avg_income'])
print(data_shapejson['features'][0]['info']['mkt_penlt'])

Mae Hong Son
83000.0
8654.0
5.9


<font face="Verdana, cursive, sans-serif" >
<h3>Save data+shape json into a javascript file</h3>
<p><code>var locationId</code> is needed for leaflets.js, hence we are going to assigned the whole json body into this variable and save it into a javascript file

In [9]:
tmp = json.dumps(data_shapejson)
output_var = 'var locationId = [' + tmp + ']'

In [10]:
print("writing js to {}".format(output_js))
with open(output_js, 'w') as the_file:
    the_file.write(output_var)

writing js to ./data/th_geo.js


<font face="Verdana, cursive, sans-serif" >
<br>
<H2>Part 2 : Leaflets.js Configuration </H2>

<p></p>
An html  is provided; of which all actions has been implemented in <code>script/setup_past_1.js</code> and <code>script/setup_part_2.js</code>
<br><code>MapViz - Thailand.html</code>
<img src="./image/mapviz_th.png" >

<p></p>
This section shows how all javascripts are imported
<img src="./image/import_html.png" >

<p></p>
This section shows how to add new attributes in the dropdown lists and how to add more color themes
<img src="./image/color_scale.png" >

Note that, the scales of each themes are calculated dynamically based on the input <code>max,min</code>
<br>Should you need to add more themes, please edit <code>script/setup_past_1.js</code>

<p></p>
To modify the top right title and to configurate which data to show on the top right section
<img src="./image/hover.png" >
<img src="./image/show_hover.png" >

<p></p>
To modify popup onclick icon and onclick polygon
<img src="./image/popup.png" >

<p></p>
Finally, this shows how the geojson data get loaded into the html
<img src="./image/locationId.png" >
